In [ ]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

def masks_to_boxes(mask_path, image_id, annotation_id, img):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    annotations = []
    scale_x = img.shape[1] / mask.shape[1]
    scale_y = img.shape[0] / mask.shape[0]


    _, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    kernel = np.ones((2, 2), np.uint8)
    eroded_mask = cv2.erode(binary_mask, kernel, iterations=2)
    processed_mask = cv2.morphologyEx(eroded_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(processed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        if w * h < 5:  
            continue
        x, y, w, h = max(0, x), max(0, y), min(w, mask.shape[1] - x), min(h, mask.shape[0] - y)

        annotations.append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": 1, 
            "bbox": [int(x * scale_x), int(y * scale_y), int(w * scale_x), int(h * scale_y)],
            "area": int(w * scale_x * h * scale_y),
        })
        annotation_id += 1

    return annotations, annotation_id


In [ ]:


image_id = 1
annotation_id = 1
dataset = []

root_dir = "/home/walke/college/cv/ass3/CV_A3/Q2/detection/train/"
images_dir = os.path.join(root_dir, "images")
masks_dir = os.path.join(root_dir, "masks")

for file_name in sorted(os.listdir(images_dir)):
    if file_name.endswith(".png"):
        img_path = os.path.join(images_dir, file_name)
        mask_path = os.path.join(masks_dir, file_name)

        if os.path.exists(mask_path):
            img = cv2.imread(img_path)
            height, width, _ = img.shape

            image_info = {
                "id": image_id,
                "file_name": img_path,
                "mask_path": mask_path,
                "width": width,
                "height": height
            }

            img_annotations, annotation_id = masks_to_boxes(mask_path, image_id, annotation_id, img)

            dataset.append({
                "image": image_info,
                "annotations": img_annotations
            })

            image_id += 1

output_json = "/home/walke/college/cv/ass3/assignment-3-walkeee3/q2/q2.json"
with open(output_json, "w") as f:
    json.dump(dataset, f, indent=4)

print(f"Annotations saved to {output_json}")


In [ ]:
import os
import json
import cv2
import matplotlib.pyplot as plt

def visualize(path, image_id):
    with open(path, "r") as f:
        data = json.load(f)

    entry = None
    for item in data:
        if item["image"]["id"] == image_id:
            entry = item
            break  
    
    if entry is None:
        print(f"no image with this id")
        return
    
    image_info = entry["image"]
    img_path = image_info["file_name"]
    

    mask_path = image_info["mask_path"]

    if not os.path.exists(img_path):
        print(f"Image file not found: {img_path}")
        return
    
    if not os.path.exists(mask_path):
        print(f"Mask file not found: {mask_path}")
        return

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  

    for ann in entry["annotations"]:
        x, y, w, h = ann["bbox"]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 3)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Image ID: {image_id}")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap="gray")
    plt.title(f"Mask for Image ID: {image_id}")
    plt.axis("off")

    plt.show()

    del img
    del mask
    cv2.destroyAllWindows()

visualize(output_json, image_id=189)
visualize(output_json, image_id=19)
visualize(output_json, image_id=7)
visualize(output_json, image_id=9)
visualize(output_json, image_id=59)


In [ ]:

import torch
import torchvision.transforms.functional as F
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class FruitDataset(Dataset):
    def __init__(self, json_path, transform=None):
        with open(json_path, "r") as f:
            self.data = json.load(f)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        image_info = entry["image"]
        annotations = entry["annotations"]

        img = cv2.imread(image_info["file_name"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (image_info["width"], image_info["height"]))
        img = F.to_tensor(img)
        img = F.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        boxes = []
        areas = []
        labels = []

        for annotation in annotations:
            x, y, w, h = annotation["bbox"]  
            category_id = annotation["category_id"]  
            
            boxes.append([x, y, x + w, y + h])
            areas.append(annotation["area"])
            labels.append(category_id) 

        boxes = torch.as_tensor(boxes, dtype=torch.int64)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
        }

        # Apply transformations (if any)
        if self.transform:
            img, target = self.transform(img, target)

        return img, target


In [ ]:
# https://pytorch.org/vision/main/generated/torchvision.transforms.ColorJitter.html
from PIL import Image

def color_jitter(image: Image.Image, target: dict,
                         brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1):
    color_jitter = transforms.ColorJitter(brightness=brightness,
                                 contrast=contrast,
                                 saturation=saturation,
                                 hue=hue)
    jittered_image = color_jitter(image)
    return jittered_image, target


import random

def compose_transforms(image, target):

    # image_np = (image.mul(255).permute(1, 2, 0).cpu().numpy()).astype(np.uint8)

    # pil_img = Image.fromarray(image_np)

    # if random.random() < 0.3:     
    #     pil_img, target = color_jitter(pil_img, target, brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05)

    # image = F.to_tensor(pil_img)

    return image, target

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

batch_size = 1

dataset = FruitDataset(output_json, compose_transforms)

train_size = int(0.9* len(dataset))

val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# train_dataset_s, val_dataset_s = torch.utils.data.random_split(val_dataset, [int(0.8* len(val_dataset)), len(val_dataset) - int(0.8* len(val_dataset))])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


len(dataset)


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:

with open(output_json, "r") as f:
    data = json.load(f)

scales = []
aspect_ratios = []
positions_x = []
positions_y = []

for entry in data:
    img_width = entry["image"]["width"]
    img_height = entry["image"]["height"]

    for annotation in entry["annotations"]:
        x, y, w, h = annotation["bbox"]
            
        scale = (w * h) / (img_width * img_height)
        scales.append(scale)

        if h > 0: 
            aspect_ratio = w / h
            aspect_ratios.append(aspect_ratio)

        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        positions_x.append(x_center)
        positions_y.append(y_center)


fig, axes = plt.subplots(1, 3, figsize=(18, 5))


axes[0].hist(scales, bins=30, alpha=0.7)
axes[0].set_title("Distribution of Scales")
axes[0].set_xlabel("Normalized Scale (w * h / image area)")
axes[0].set_ylabel("Frequency")

axes[1].hist(aspect_ratios, bins=30, alpha=0.7)
axes[1].set_title("Distribution of Aspect Ratios")
axes[1].set_xlabel("Aspect Ratio (w / h)")
axes[1].set_ylabel("Frequency")

axes[2].scatter(positions_x, positions_y, alpha=0.5, s=10)
axes[2].set_title("Distribution of Bounding Box Positions")
axes[2].set_xlabel("Normalized X Center")
axes[2].set_ylabel("Normalized Y Center")


plt.tight_layout()
plt.show()




# **Plot 1**
Most bounding boxes are very small relative to the image area so we should use smaller anchor sizes for better detection.
# **Plot 2**
Most aspect ratios are around 1 most the boxes are almost square.
This justifies using aspect ratios like (0.8, 1.0, 1.2) instead of extreme values.
# **Plot 3**
The bounding boxes are evenly distributed across the image, so no major positional bias.

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torch.nn as nn


backbone = torchvision.models.resnet34(weights="IMAGENET1K_V1")
modules = list(backbone.children())[:-2] 
backbone = nn.Sequential(*modules)
backbone.out_channels = 512 


anchor_sizes = ((16, 32, 64, 128),)  
aspect_ratios = ((0.8, 1.0, 1.2),)  

anchor_generator = AnchorGenerator(sizes=anchor_sizes, aspect_ratios=aspect_ratios)

model = FasterRCNN(
    backbone=backbone,
    num_classes=2,  
    rpn_anchor_generator=anchor_generator,
    box_detections_per_img=30,
)

model = model.to(device)

print(model)


In [ ]:

IMAGENET_MEAN = np.array([0.485, 0.456, 0.406])
IMAGENET_STD = np.array([0.229, 0.224, 0.225])
def unnormalize_image(image):

    image = image.permute(1, 2, 0).cpu().numpy()  
    image = (image * IMAGENET_STD) + IMAGENET_MEAN 
    image = (image * 255).astype(np.uint8)  
    return image

def visualize_predictions(images_list, predictions_list):
    num_samples = min(5, len(images_list))  

    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    if num_samples == 1:
        axes = [axes]  

    for ax, img, pred in zip(axes, images_list[:num_samples], predictions_list[:num_samples]):

        img = unnormalize_image(img)
        img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        ax.imshow(img)  
        ax.axis("off")

        for box in pred["boxes"]:
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor="red", linewidth=2)
            ax.add_patch(rect)

    plt.show()

In [ ]:
from tqdm import tqdm
import gc
import torch

def train_loop(dataloader, model, optimizer, device):
    model.train()  
    total_loss = 0.0
    num_batches = len(dataloader)

    for images, targets in tqdm(dataloader, desc="Training", total=num_batches):

        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        targets = [t for t in targets if t["boxes"].shape[0] > 0]
        if len(targets) == 0:
            continue


        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        total_loss += loss_value

        losses.backward()
        optimizer.step()

    torch.cuda.empty_cache()
    gc.collect()

    avg_loss = total_loss / num_batches
    print(f"Training: Average Loss: {avg_loss:.6f}")

def val_loop(dataloader, model, device):
    model.eval()  
    predictions_list = []
    images_list = []
    total_loss = 0.0
    num_batches = len(dataloader)

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Validation", total=num_batches):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            valid_data = [(img, tgt) for img, tgt in zip(images, targets) if tgt["boxes"].shape[0] > 0]
            if not valid_data:
                continue
            images, targets = zip(*valid_data) 


            model.train()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
            total_loss += loss_value

            model.eval()
            predictions = model(images)  
            predictions = [{k: v.cpu() for k, v in p.items()} for p in predictions]

            for img, pred in zip(images, predictions):
                images_list.append(img.cpu())
                predictions_list.append(pred)

    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    print(f"Validation: Average Loss: {avg_loss:.6f}")

    if len(images_list) > 0:
        visualize_predictions(images_list, predictions_list)

    del images_list
    del predictions_list





In [ ]:
import torch.optim as optim

learning_rate = 1e-3
epochs = 10

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_dataloader, model, optimizer, device)
    val_loop(val_dataloader, model, device)

    
print("Training complete!")

In [ ]:


class testDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.folder_path, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")  # Ensure 3-channel image

        if self.transform:
            image = self.transform(image)
        return image 

# Define Image Transformations
transform = transforms.Compose([
    transforms.ToTensor(),   
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_folder = "/home/walke/college/cv/ass3/CV_A3/Q2/detection/test/images"  


test_dataset = testDataset(test_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)



In [ ]:

from torchvision.ops import nms

def visualize_test(images_list, predictions_list, num_samples=5):
    fig, axes = plt.subplots(1, min(num_samples, len(images_list)), figsize=(15, 5))

    if len(images_list) == 1:
        axes = [axes] 

    for ax, img, pred in zip(axes, images_list[:num_samples], predictions_list[:num_samples]):
        img = img.cpu()  
        ax.imshow(img.permute(1, 2, 0).numpy())  
        ax.axis("off")

        for box in pred["boxes"]:
            box = box.cpu().numpy() 
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor="red", linewidth=2)
            ax.add_patch(rect)

    plt.show()


def inference_with_nms(model, dataloader, device, iou_threshold=0.5, score_threshold=0.5):

    model.eval()
    results = []
    images_list = []

    with torch.no_grad():
        for images in dataloader:
            images = [img.to(device) for img in images]
            predictions = model(images)  

            for i, pred in enumerate(predictions):
                boxes = pred["boxes"].detach().cpu()
                scores = pred["scores"].detach().cpu()
                labels = pred["labels"].detach().cpu()

                keep_indices = nms(boxes, scores, iou_threshold)

                filtered_boxes = boxes[keep_indices]
                filtered_scores = scores[keep_indices]
                filtered_labels = labels[keep_indices]

                valid_indices = filtered_scores >= score_threshold
                final_boxes = filtered_boxes[valid_indices]
                final_scores = filtered_scores[valid_indices]
                final_labels = filtered_labels[valid_indices]

                results.append({
                    "boxes": final_boxes,
                    "scores": final_scores,
                    "labels": final_labels,
                })

                images_list.append(images[i].cpu())  

    if len(images_list) > 0:
        visualize_test(images_list, results)  

inference_with_nms(model, test_dataloader, device, iou_threshold=0.5, score_threshold=0.5)



nms is required to supress multiple fruit detection which cause overlapping detections
- Anchor boxes overlap
- Fruits appear in clusters
- Keeps only the most confident bounding box per fruit.

multiple overlapping fruit detections acan be dealt by using techniques like wbf which take the average box over all the overlapping prodections

we can also use soft-nms which decays the iou instead

iou threshholds
0.3 - 0.4	Too low → false positives (too many boxes)
0.5 - 0.6  Balanced → Good for clustered objects like fruits
0.7 - 0.9	Too high → true positives removed (missed detections)

for data augmentation i had added the color jittering transform.

Different Confidence Thresholds
0.2 - 0.4	Too low → Many false positives
0.5 - 0.7 	Balanced → Best trade-off
0.8 - 0.9	Too high → Misses true detections

more resolutions leads to longer processing time but smaller objects like fruits are detected better


In [ ]:


def apply_color_jitter(images):
    transform = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    return [transform(img) for img in images]

def comp_loop(dataloader, model, device, conf_threshold=0.5, use_augmentation=False):
    model.train()
    total_loss = 0.0
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc=f"Validation (Conf={conf_threshold}, Aug={use_augmentation})", total=num_batches):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            if use_augmentation:
                images = apply_color_jitter(images)
            
            valid_data = [(img, tgt) for img, tgt in zip(images, targets) if tgt["boxes"].shape[0] > 0]
            if not valid_data:
                continue
            images, targets = zip(*valid_data)
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            total_loss += losses.item()
    
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    print(f"Validation Results -> Conf: {conf_threshold}, Aug: {use_augmentation}, Avg Loss: {avg_loss:.6f}")
    return avg_loss

# Experiment with different configurations
configurations = [
    {"conf_threshold": 0.5, "use_augmentation": False},
    {"conf_threshold": 0.5, "use_augmentation": True},
    {"conf_threshold": 0.3, "use_augmentation": False},
    {"conf_threshold": 0.7, "use_augmentation": False},
    {"conf_threshold": 0.9, "use_augmentation": True},
]

results = {}
for config in configurations:
    loss = comp_loop(val_dataloader, model, device, **config)
    results[str(config)] = loss

# Compare and analyze the results
print("Final Comparison:")
for key, value in results.items():
    print(f"{key}: Avg Loss = {value:.6f}")


In [ ]:
import torch
from tqdm import tqdm

def metric_loop(dataloader, model, device):
    model.eval()  
    predictions_list = []
    ground_truths_list = []
    images_list = []
    total_loss = 0.0
    num_batches = len(dataloader)

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Validation", total=num_batches):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            valid_data = [(img, tgt) for img, tgt in zip(images, targets) if tgt["boxes"].shape[0] > 0]
            if not valid_data:
                continue  
            images, targets = zip(*valid_data)

            model.train()
            loss_dict = model(images, targets)
            if not isinstance(loss_dict, dict):  
                print(f"Unexpected loss output type: {type(loss_dict)}")  
                continue  
            losses = sum(loss for loss in loss_dict.values())
            total_loss += losses.item()

            model.eval()
            predictions = model(images)  
            predictions = [{k: v.cpu() for k, v in p.items()} for p in predictions]

            for img, pred, tgt in zip(images, predictions, targets):
                images_list.append(img.cpu())
                
                predictions_list.append({
                    "boxes": pred["boxes"], 
                    "scores": pred["scores"], 
                    "labels": pred["labels"]
                })
                
                ground_truths_list.append({
                    "boxes": tgt["boxes"].cpu(), 
                    "labels": tgt["labels"].cpu()
                })

    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    print(f"Metric: Average Loss: {avg_loss:.6f}")

    if len(images_list) > 0:
        visualize_predictions(images_list, predictions_list)

    del images_list

    return avg_loss, predictions_list, ground_truths_list 


In [ ]:

from collections import defaultdict

def apply_nms(predictions, iou_threshold=0.5):
    filtered_predictions = []

    for pred in predictions:
        if len(pred["boxes"]) == 0:
            filtered_predictions.append(pred) 
            continue

        boxes = pred["boxes"]
        scores = pred["scores"]
        labels = pred["labels"]

        keep_indices = nms(boxes, scores, iou_threshold)

        filtered_predictions.append({
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        })

    return filtered_predictions

def compute_iou(box1, box2):

    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    intersection_area = inter_width * inter_height

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0

    return iou


def compute_ap(precision, recall):
    recall = np.concatenate(([0.0], recall, [1.0]))
    precision = np.concatenate(([0.0], precision, [0.0]))

    for i in range(len(precision) - 2, -1, -1):
        precision[i] = max(precision[i], precision[i + 1])

    indices = np.where(recall[1:] != recall[:-1])[0]
    ap = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
    return ap

def compute_map(predictions, ground_truths, iou_thresholds=[0.5, 0.75]):
    results = {}
    for iou_thresh in iou_thresholds:
        all_precisions, all_recalls = [], []
        for gt, pred in zip(ground_truths, predictions):
            gt_boxes, gt_labels = gt["boxes"], gt["labels"]
            pred_boxes, pred_scores, pred_labels = pred["boxes"], pred["scores"], pred["labels"]

            if len(gt_boxes) == 0 and len(pred_boxes) == 0:
                continue

            tp, fp = 0, 0
            assigned_gt = set()
            sorted_indices = np.argsort(-pred_scores.cpu().numpy())  

            for idx in sorted_indices:
                best_iou, best_gt_idx = 0, -1
                for j, gt_box in enumerate(gt_boxes):
                    if j in assigned_gt or gt_labels[j] != pred_labels[idx]:
                        continue

                    iou = compute_iou(pred_boxes[idx].cpu().numpy(), gt_box.cpu().numpy())
                    if iou > best_iou:
                        best_iou, best_gt_idx = iou, j

                if best_iou > iou_thresh:
                    tp += 1
                    assigned_gt.add(best_gt_idx)
                else:
                    fp += 1

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / len(gt_boxes) if len(gt_boxes) > 0 else 0
            all_precisions.append(precision)
            all_recalls.append(recall)

        mean_ap = compute_ap(np.array(all_precisions), np.array(all_recalls))
        results[f"mAP@{iou_thresh}"] = mean_ap

    return results
avg_loss, predictions_list, ground_truths_list = metric_loop(val_dataloader, model, device)

predictions_list = apply_nms(predictions_list)
mAP_results = compute_map(predictions_list, ground_truths_list)
print("mAP Scores:", mAP_results)


In [ ]:
def analyze_fp_fn(predictions, ground_truths, iou_threshold=0.5):
 
    fp_count, fn_count = 0, 0

    for gt, pred in zip(ground_truths, predictions):
        gt_boxes, gt_labels = gt["boxes"], gt["labels"]
        pred_boxes, pred_scores, pred_labels = pred["boxes"], pred["scores"], pred["labels"]

        matched_gt = set()

        for pred_box, pred_label in zip(pred_boxes, pred_labels):
            best_iou, best_gt_idx = 0, -1
            for j, gt_box in enumerate(gt_boxes):
                if j in matched_gt or gt_labels[j] != pred_label:
                    continue

                iou = compute_iou(pred_box.cpu().numpy(), gt_box.cpu().numpy())
                if iou > best_iou:
                    best_iou, best_gt_idx = iou, j

            if best_iou > iou_threshold:
                matched_gt.add(best_gt_idx)
            else:
                fp_count += 1  

        fn_count += len(gt_boxes) - len(matched_gt)  

    print(f"False Positives: {fp_count}")
    print(f"False Negatives: {fn_count}")
    
predictions_list = apply_nms(predictions_list)
analyze_fp_fn(predictions_list, ground_truths_list, iou_threshold=0.5)


- Impact of occlusion on detection accuracy
it would affect it a lot as there would be overlapping boxes and the size of the fruits is very small if it is covered by another object already less information is reduced even more

- Effect of fruit color variations and lighting conditions
the fruit color doesnt matter much but brighter colors would be better as the background of the trees is mostly green the mask does not detect the fruits in the darker reggions in the trainid set.

- Performance on densely clustered fruits versus isolated fruits
there are many overlapping boxxes for densely cluster fruits which leads to false positives due to duplicates.

- Model’s behavior with fruits at different scales
this model classifies small fruits wtr to the rest of the image because of how the dataset is

### **Potential Modifications**  

#### **1. Detecting Multiple Fruit Types**  
- Increase `num_classes`, expand dataset, and use a deeper backbone (ResNet-50, EfficientNet).  
- Optimize anchor boxes and implement class-aware NMS to improve multi-fruit detection.  

#### **2. Robustness to Environmental Variations**  
- Use data augmentation (brightness, contrast, occlusion) and diverse training data.  
- Apply adaptive confidence thresholds and preprocessing (CLAHE, histogram equalization).  

#### **3. Handling Growth Stages & Ripeness**  
- Train for ripeness classification using HSV color space and texture-based features.  
- Adjust anchor sizes for multi-scale detection and use temporal consistency for ripeness.  